Reminder:  Please see VISUAL_OVERVIEW.md and DATA_INVENTORY.md for more information

Step 1:  Get the Data File from the /INPUTS directory.  We are looking for .zip files

## Setup: Create Virtual Environment (Local Dev Only)
Run this cell to create and activate a Python virtual environment for local development.

In [1]:
import subprocess
import sys
import os

# Create virtual environment
venv_dir = './venv'
subprocess.run([sys.executable, '-m', 'venv', venv_dir], check=True)
print(f'Virtual environment created at {venv_dir}')

# Determine pip executable path
if os.name == 'nt':  # Windows
    pip_executable = os.path.join(venv_dir, 'Scripts', 'pip')
else:  # macOS/Linux
    pip_executable = os.path.join(venv_dir, 'bin', 'pip')

# Install required packages
packages = ['pyyaml', 'pydantic', 'sqlalchemy', 'pyodbc', 'pandas', 'numpy']
subprocess.run([pip_executable, 'install'] + packages, check=True)

print('Virtual environment setup complete!')
print(f'To activate on Windows: .\\{venv_dir}\\Scripts\\activate')
print(f'To activate on macOS/Linux: source {venv_dir}/bin/activate')

Virtual environment created at ./venv
Virtual environment setup complete!
To activate on Windows: .\./venv\Scripts\activate
To activate on macOS/Linux: source ./venv/bin/activate


In [8]:
# Step 1: Locate the .zip input file from /INPUTS directory
import os
import zipfile
from pathlib import Path

# Navigate to /INPUTS directory (one level up from Notebooks)
notebook_dir = Path.cwd()
inputs_dir = Path('..') / 'INPUTS'  # One level up to workspace root, then INPUTS
inputs_dir = inputs_dir.resolve()

# Create working directory for extracted data
work_dir = Path('./data').resolve()
work_dir.mkdir(exist_ok=True)

# Find .zip files in INPUTS
if inputs_dir.exists():
    zip_files = list(inputs_dir.glob('*.zip'))
    print(f'✓ Found INPUTS directory at: {inputs_dir}')
    print(f'\nZip files found:')
    for zip_file in zip_files:
        print(f'  - {zip_file.name}')
    
    # Extract the first (or only) zip file
    if zip_files:
        zip_path = zip_files[0]
        extract_to = work_dir / zip_path.stem
        extract_to.mkdir(exist_ok=True)
        
        print(f'\nExtracting {zip_path.name}...')
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_to)
        print(f'✓ Extracted to: {extract_to}')
    else:
        print('✗ No .zip files found')
else:
    print(f'✗ INPUTS directory not found at: {inputs_dir}')

✓ Found INPUTS directory at: C:\Projects\Clients\GelSight\Gelsight Application Folder\INPUTS

Zip files found:
  - GelSightAnalysis.zip

Extracting GelSightAnalysis.zip...
✓ Extracted to: C:\Projects\Clients\GelSight\Gelsight Application Folder\Notebooks\data\GelSightAnalysis


## Step 2: Parse YAML Files and Convert to JSON
Extract scan metadata, analysis results, and calibration data from the zip contents and organize into bronze layer structure.

In [19]:
import yaml
import json
import shutil
import math
from pathlib import Path
from datetime import datetime, date, time

# Helper function to convert non-serializable types for JSON
def convert_nan(obj):
    """Recursively convert NaN, Inf, datetime, and other non-JSON-serializable values"""
    if isinstance(obj, dict):
        return {k: convert_nan(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [convert_nan(item) for item in obj]
    elif isinstance(obj, (datetime, date, time)):
        return obj.isoformat()  # Convert datetime to ISO format string
    elif isinstance(obj, float):
        if math.isnan(obj) or math.isinf(obj):
            return None
    return obj

# Define paths
work_dir = Path('./data').resolve()
outputs_dir = Path('./OUTPUTS').resolve()
outputs_dir.mkdir(exist_ok=True)

bronze_scans = outputs_dir / 'customer01' / 'bronze' / 'scans'
bronze_calibrations = outputs_dir / 'customer01' / 'bronze' / 'calibrations'
bronze_scans.mkdir(parents=True, exist_ok=True)
bronze_calibrations.mkdir(parents=True, exist_ok=True)

# Find the extracted GelSightAnalysis folder
# Handle nested GelSightAnalysis/GelSightAnalysis structures
gelsight_dir = work_dir / 'GelSightAnalysis'

# Check if there's a nested GelSightAnalysis folder
if gelsight_dir.exists():
    nested_gel = gelsight_dir / 'GelSightAnalysis'
    if nested_gel.exists():
        gelsight_dir = nested_gel

# If still not found, try glob pattern
if not gelsight_dir.exists():
    extracted_folders = list(work_dir.glob('**/GelSightAnalysis'))
    if extracted_folders:
        # Use the deepest nested one
        gelsight_dir = extracted_folders[-1]

if gelsight_dir.exists():
    print(f'✓ Processing GelSightAnalysis folder: {gelsight_dir}\n')
    
    # Process each analysis type folder (DefectDetection, HoleDiameter, etc.)
    analysis_folders = [d for d in gelsight_dir.iterdir() if d.is_dir()]
    
    if not analysis_folders:
        print(f'✗ No analysis folders found in {gelsight_dir}')
        print(f'Contents: {list(gelsight_dir.iterdir())}')
    
    for analysis_folder in sorted(analysis_folders):
        analysis_type = analysis_folder.name
        print(f'\nProcessing: {analysis_type}')
        
        # Create folder for this scan
        scan_id = analysis_type
        scan_folder = bronze_scans / scan_id
        scan_folder.mkdir(exist_ok=True)
        
        # Extract scan.yaml
        scan_yaml = analysis_folder / 'scan.yaml'
        if scan_yaml.exists():
            with open(scan_yaml, 'r') as f:
                scan_data = yaml.safe_load(f)
            # Save as JSON (convert non-serializable types)
            scan_json = scan_folder / 'scan_metadata.json'
            with open(scan_json, 'w') as f:
                json.dump(convert_nan(scan_data), f, indent=2)
            print(f'  ✓ scan.yaml -> scan_metadata.json')
        else:
            print(f'  ✗ scan.yaml not found at {scan_yaml}')
        
        # Extract analysis/scancontext.yaml
        analysis_yaml = analysis_folder / 'analysis' / 'scancontext.yaml'
        if analysis_yaml.exists():
            with open(analysis_yaml, 'r') as f:
                analysis_data = yaml.safe_load(f)
            # Save as JSON (convert non-serializable types)
            analysis_json = scan_folder / 'analysis_results.json'
            with open(analysis_json, 'w') as f:
                json.dump(convert_nan(analysis_data), f, indent=2)
            print(f'  ✓ scancontext.yaml -> analysis_results.json')
        else:
            print(f'  ✗ analysis/scancontext.yaml not found')
        
        # Extract calibration file (Calib-*.yaml)
        calib_files = list(analysis_folder.glob('Calib-*.yaml'))
        for calib_file in calib_files:
            with open(calib_file, 'r') as f:
                calib_data = yaml.safe_load(f)
            # Save as JSON in calibrations folder (convert non-serializable types)
            calib_id = calib_file.stem
            calib_folder = bronze_calibrations / calib_id
            calib_folder.mkdir(exist_ok=True)
            calib_json = calib_folder / 'calib_metadata.json'
            with open(calib_json, 'w') as f:
                json.dump(convert_nan(calib_data), f, indent=2)
            print(f'  ✓ {calib_file.name} -> JSON')
        
        # Copy .tmd files if they exist
        tmd_files = list(analysis_folder.glob('*.tmd'))
        heightmaps_dir = scan_folder / 'heightmaps'
        if tmd_files:
            heightmaps_dir.mkdir(exist_ok=True)
            for tmd_file in tmd_files:
                shutil.copy(tmd_file, heightmaps_dir / tmd_file.name)
            print(f'  ✓ Copied {len(tmd_files)} .tmd file(s)')
        
        # Copy PNG images if they exist
        png_files = list(analysis_folder.glob('*.png'))
        if png_files:
            images_dir = scan_folder / 'images'
            images_dir.mkdir(exist_ok=True)
            for png_file in png_files:
                shutil.copy(png_file, images_dir / png_file.name)
            print(f'  ✓ Copied {len(png_files)} PNG image(s)')
        
        # Copy raw YAML files for audit trail
        raw_yaml_dir = scan_folder / 'raw_yaml'
        raw_yaml_dir.mkdir(exist_ok=True)
        
        # Copy scan.yaml
        if scan_yaml.exists():
            shutil.copy(scan_yaml, raw_yaml_dir / 'scan.yaml')
            print(f'  ✓ Preserved raw scan.yaml')
        
        # Copy scancontext.yaml
        if analysis_yaml.exists():
            shutil.copy(analysis_yaml, raw_yaml_dir / 'scancontext.yaml')
            print(f'  ✓ Preserved raw scancontext.yaml')
        
        # Copy calibration files
        for calib_file in calib_files:
            shutil.copy(calib_file, raw_yaml_dir / calib_file.name)
    
    print(f'\n\n✓ Data extraction complete!')
    print(f'\nOutput structure:')
    print(f'  Bronze scans: {bronze_scans}')
    print(f'  Bronze calibrations: {bronze_calibrations}')
    print(f'\nEach scan folder contains:')
    print(f'  - scan_metadata.json')
    print(f'  - analysis_results.json')
    print(f'  - images/ (PNG files)')
    print(f'  - heightmaps/ (.tmd files)')
    print(f'  - raw_yaml/ (original YAML files for audit)')
else:
    print(f'✗ GelSightAnalysis folder not found')
    print(f'Searched in: {work_dir}')
    print(f'Contents: {list(work_dir.iterdir()) if work_dir.exists() else "work_dir does not exist"}')

✓ Processing GelSightAnalysis folder: C:\Projects\Clients\GelSight\Gelsight Application Folder\Notebooks\data\GelSightAnalysis\GelSightAnalysis


Processing: DefectDetection
  ✓ scan.yaml -> scan_metadata.json
  ✓ scancontext.yaml -> analysis_results.json
  ✓ Calib-4E07-XNJU_20250917_1255.yaml -> JSON
  ✓ Copied 1 .tmd file(s)
  ✓ Copied 15 PNG image(s)
  ✓ Preserved raw scan.yaml
  ✓ Preserved raw scancontext.yaml

Processing: HoleDiameter
  ✓ scan.yaml -> scan_metadata.json
  ✓ scancontext.yaml -> analysis_results.json
  ✓ Calib-4E07-XNJU_20250917_1255.yaml -> JSON
  ✓ Copied 1 .tmd file(s)
  ✓ Copied 15 PNG image(s)
  ✓ Preserved raw scan.yaml
  ✓ Preserved raw scancontext.yaml

Processing: Offset
  ✓ scan.yaml -> scan_metadata.json
  ✓ scancontext.yaml -> analysis_results.json
  ✓ Calib-4E07-XNJU_20250917_1255.yaml -> JSON
  ✓ Copied 1 .tmd file(s)
  ✓ Copied 15 PNG image(s)
  ✓ Preserved raw scan.yaml
  ✓ Preserved raw scancontext.yaml

Processing: PitDetection
  ✓ scan.yaml -> sc

## Step 3: Load Bronze Data into DataFrames and Transform to Silver Layer
Combine scan metadata, analysis results, and calibration data into unified DataFrames and save to silver layer.

In [22]:
import pandas as pd
import json
from pathlib import Path
from datetime import datetime

# Define paths
outputs_dir = Path('./OUTPUTS').resolve()
bronze_scans = outputs_dir / 'customer01' / 'bronze' / 'scans'
bronze_calibrations = outputs_dir / 'customer01' / 'bronze' / 'calibrations'

silver_scans = outputs_dir / 'customer01' / 'silver' / 'scans'
silver_calibrations = outputs_dir / 'customer01' / 'silver' / 'calibrations'
silver_scans.mkdir(parents=True, exist_ok=True)
silver_calibrations.mkdir(parents=True, exist_ok=True)

print(f'✓ Loading bronze layer data from: {bronze_scans}\n')

# Load all scan metadata and analysis results into DataFrames
scan_records = []
analysis_records = []

if bronze_scans.exists():
    scan_folders = [d for d in bronze_scans.iterdir() if d.is_dir()]
    
    for scan_folder in sorted(scan_folders):
        scan_id = scan_folder.name
        
        # Load scan metadata
        scan_json = scan_folder / 'scan_metadata.json'
        if scan_json.exists():
            try:
                with open(scan_json, 'r') as f:
                    scan_data = json.load(f)
                # Add scan_id for later joining
                scan_data['scan_id'] = scan_id
                scan_records.append(scan_data)
                print(f'  ✓ Loaded scan_metadata.json from {scan_id}')
            except json.JSONDecodeError as e:
                print(f'  ✗ Invalid JSON in {scan_id}/scan_metadata.json: {e}')
        
        # Load analysis results
        analysis_json = scan_folder / 'analysis_results.json'
        if analysis_json.exists():
            try:
                with open(analysis_json, 'r') as f:
                    analysis_data = json.load(f)
                # Add scan_id for joining
                analysis_data['scan_id'] = scan_id
                analysis_records.append(analysis_data)
                print(f'  ✓ Loaded analysis_results.json from {scan_id}')
            except json.JSONDecodeError as e:
                print(f'  ✗ Invalid JSON in {scan_id}/analysis_results.json: {e}')
    
    # Create DataFrames with error handling for nested structures
    try:
        # Flatten and create scans DataFrame
        scans_df = pd.json_normalize(scan_records) if scan_records else pd.DataFrame()
        print(f'\n✓ Created scans DataFrame: {len(scans_df)} rows, {len(scans_df.columns)} columns')
        print(f'  Columns: {list(scans_df.columns)[:5]}...')  # Show first 5 columns
        
        # Flatten and create analysis DataFrame
        analysis_df = pd.json_normalize(analysis_records) if analysis_records else pd.DataFrame()
        print(f'✓ Created analysis DataFrame: {len(analysis_df)} rows, {len(analysis_df.columns)} columns')
        print(f'  Columns: {list(analysis_df.columns)[:5]}...')
        
        # Merge scans and analysis on scan_id if both exist
        if not scans_df.empty and not analysis_df.empty:
            merged_df = scans_df.merge(analysis_df, on='scan_id', how='left', suffixes=('_scan', '_analysis'))
            print(f'\n✓ Merged scans + analysis: {len(merged_df)} rows')
        else:
            merged_df = scans_df if not scans_df.empty else analysis_df
        
        # Save silver layer data as CSV
        if not merged_df.empty:
            # Save as CSV (readable, portable)
            silver_scans_csv = silver_scans / 'scans_merged.csv'
            merged_df.to_csv(silver_scans_csv, index=False)
            print(f'  ✓ Saved to: {silver_scans_csv}')
            
            # Save summary statistics
            summary = {
                'total_records': len(merged_df),
                'total_columns': len(merged_df.columns),
                'columns': list(merged_df.columns),
                'dtypes': merged_df.dtypes.astype(str).to_dict(),
                'generated_at': datetime.now().isoformat()
            }
            summary_json = silver_scans / 'data_summary.json'
            with open(summary_json, 'w') as f:
                json.dump(summary, f, indent=2)
            print(f'  ✓ Saved summary to: {summary_json}')
    
    except Exception as e:
        print(f'\n✗ Error during DataFrame creation: {e}')
        print(f'  This may indicate nested/complex structures. Raw data still available in bronze layer.')

# Load and process calibrations
calibration_records = []
if bronze_calibrations.exists():
    calib_folders = [d for d in bronze_calibrations.iterdir() if d.is_dir()]
    
    print(f'\n✓ Loading calibration data from: {bronze_calibrations}')
    for calib_folder in sorted(calib_folders):
        calib_id = calib_folder.name
        calib_json = calib_folder / 'calib_metadata.json'
        if calib_json.exists():
            with open(calib_json, 'r') as f:
                calib_data = json.load(f)
            calib_data['calib_id'] = calib_id
            calibration_records.append(calib_data)
            print(f'  ✓ Loaded {calib_id}')
    
    if calibration_records:
        calib_df = pd.json_normalize(calibration_records)
        print(f'\n✓ Created calibrations DataFrame: {len(calib_df)} rows, {len(calib_df.columns)} columns')
        
        # Save silver layer calibrations as CSV
        silver_calib_csv = silver_calibrations / 'calibrations.csv'
        calib_df.to_csv(silver_calib_csv, index=False)
        print(f'  ✓ Saved to: {silver_calib_csv}')

print(f'\n✓ Silver layer transformation complete!')
print(f'\nOutput location: {outputs_dir / "customer01" / "silver"}')
print(f'  scans_merged.csv - Merged scan + analysis data')
print(f'  calibrations.csv - Calibration data')
print(f'  data_summary.json - Metadata about the merged data')

✓ Loading bronze layer data from: C:\Projects\Clients\GelSight\Gelsight Application Folder\Notebooks\OUTPUTS\customer01\bronze\scans

  ✓ Loaded scan_metadata.json from DefectDetection
  ✓ Loaded analysis_results.json from DefectDetection
  ✓ Loaded scan_metadata.json from HoleDiameter
  ✓ Loaded analysis_results.json from HoleDiameter
  ✓ Loaded scan_metadata.json from Offset
  ✓ Loaded analysis_results.json from Offset
  ✓ Loaded scan_metadata.json from PitDetection
  ✓ Loaded analysis_results.json from PitDetection
  ✓ Loaded scan_metadata.json from SurfaceRoughness
  ✓ Loaded analysis_results.json from SurfaceRoughness

✓ Created scans DataFrame: 5 rows, 38 columns
  Columns: ['version', 'guid', 'createdon', 'mmperpixel', 'sdkversion']...
✓ Created analysis DataFrame: 5 rows, 4 columns
  Columns: ['version', 'shapes', 'routines', 'scan_id']...

✓ Merged scans + analysis: 5 rows
  ✓ Saved to: C:\Projects\Clients\GelSight\Gelsight Application Folder\Notebooks\OUTPUTS\customer01\silve

## Step 4: Create Databricks Tables from Silver Layer
**Create External Tables in Unity Catalog**

### What We've Accomplished
We have successfully extracted and transformed GelSight analysis data into two storage layers:

- **Bronze Layer**: Raw data organized by analysis type (stored as JSON backup/audit trail)
  - `OUTPUTS/customer01/bronze/scans/` - Individual scan folders with JSON metadata
  - `OUTPUTS/customer01/bronze/calibrations/` - Calibration data in JSON format
  - *Keep these as-is for audit trail; no tables needed*
  
- **Silver Layer**: Merged, flattened DataFrames ready for analytics (stored as CSV)
  - `OUTPUTS/customer01/silver/scans/scans_merged.csv` - 5 scans × 42 columns (merged scan metadata + analysis results)
  - `OUTPUTS/customer01/silver/calibrations/calibrations.csv` - 2 calibrations

### Create Silver Tables in Databricks
We only need to create tables for the **silver layer CSV files** since they're already flattened and ready to query.

**Steps:**
1. **Upload** `OUTPUTS/customer01/silver/` folder to Azure storage (`abfss://customer01/silver`)
2. **Create External Tables** in Databricks Unity Catalog pointing to the CSV files:
   - `customer01_data.silver.scans` → Points to `OUTPUTS/customer01/silver/scans/scans_merged.csv`
   - `customer01_data.silver.calibrations` → Points to `OUTPUTS/customer01/silver/calibrations/calibrations.csv`
3. **Query** the tables for analytics
4. **Optionally create Gold Layer** - aggregate/clean silver data into high-level analytical tables

### Why Only Silver Tables?
- ✅ Silver CSVs are already flattened and normalized (no nested structures)
- ✅ Easy to query with standard SQL
- ✅ Bronze JSON files stay in storage as backup/audit trail (read-only reference)
- ✅ Cleaner, simpler approach for analytics

### Data Structure
| Layer | Format | Location | Purpose |
|-------|--------|----------|---------|
| Bronze | JSON | `bronze/scans/`, `bronze/calibrations/` | Raw data, audit trail, backup |
| Silver | CSV | `silver/scans/scans_merged.csv`, `silver/calibrations/calibrations.csv` | Analytics-ready, flattened data |
| Gold | (optional) | TBD | Aggregated, business-logic tables |

### Credentials & Locations
- **Databricks Workspace**: USCU-PROD-DATA-PROCESSING-01
- **Cluster**: CLU-SCAN-1
- **Unity Catalog**: `customer01_data`
- **Schemas**: `silver`, `gold` (create as needed)
- **External Location**: `uscu_silver` (credentialed with `uscu-storage-cred` managed identity)
- **Storage Account**: gelsightprodstnd01
- **Container**: customer01

### SQL Commands to Execute in Databricks
```sql
-- Create external table for silver scans
CREATE EXTERNAL TABLE IF NOT EXISTS customer01_data.silver.scans
  USING CSV
  LOCATION 'abfss://customer01/silver@gelsightprodstnd01.dfs.core.windows.net/scans/'
  WITH (header=true, inferSchema=true)

-- Create external table for silver calibrations
CREATE EXTERNAL TABLE IF NOT EXISTS customer01_data.silver.calibrations
  USING CSV
  LOCATION 'abfss://customer01/silver@gelsightprodstnd01.dfs.core.windows.net/calibrations/'
  WITH (header=true, inferSchema=true)

-- Validate the data
SELECT COUNT(*) as scan_count FROM customer01_data.silver.scans
SELECT COUNT(*) as calib_count FROM customer01_data.silver.calibrations
```

**Bronze JSON files remain in storage** at `abfss://customer01/bronze/` for backup/audit purposes but do not require Unity Catalog tables unless you specifically need to query the raw nested JSON structure.

## Step 5: Create Gold Layer - Analytics & Reporting Tables

**Transform Silver Tables into Business-Ready Analytics**

### Gold Layer Purpose
Build analytical tables optimized for:
- ✅ Master dimension of all scans (scan_registry)
- ✅ Scan-to-calibration traceability (real relationships from source data)
- ✅ Analysis-type breakdown for comparison
- ✅ Calibration usage metrics and statistics
- ✅ Dashboards and reporting

### Gold Layer Tables to Create
1. **`customer01_data.gold.scan_registry`** - Master dimension: all scan metadata with extracted calibration ID
2. **`customer01_data.gold.analysis_by_type`** - Results grouped by analysis type (DefectDetection, PitDetection, etc.)
3. **`customer01_data.gold.calibration_usage`** - Calibration usage stats: which calibrations are used, by which scans, date ranges

### Key Finding: Real Calibration Relationships! 
✅ Each scan has a `calib` column with the calibration file path
- 4 scans → Calib-4E07-XNJU_20250917_1255
- 1 scan → Calib-436U-Y58K_20250912_0928

### Next: Execute SQL Commands Below
Run the SQL cells below to create the gold layer tables in Databricks.

In [ ]:
-- Gold Layer: Scan Registry (Master Dimension Table)
-- All scans with metadata and extracted calibration ID from file path
CREATE OR REPLACE TABLE customer01_data.gold.scan_registry AS
SELECT 
    scan_id,
    guid,
    createdon,
    mmPerPixel,
    sdkversion,
    -- Extract calibration ID from calib file path (e.g., 'Calib-4E07-XNJU_20250917_1255')
    REGEXP_EXTRACT(calib, 'Calib-[^/\\]+') as calib_id,
    calib as calib_filepath
FROM customer01_data.silver.scans
ORDER BY createdon DESC;

-- Validate scan registry
SELECT * FROM customer01_data.gold.scan_registry;

In [ ]:
-- Gold Layer: Analysis By Type
-- Aggregates results by analysis type (DefectDetection, PitDetection, etc.)
CREATE OR REPLACE TABLE customer01_data.gold.analysis_by_type AS
SELECT 
    scan_id,
    createdon,
    -- Extract analysis type from scan_id (first part before any underscore/suffix)
    CASE 
        WHEN scan_id LIKE 'DefectDetection%' THEN 'Defect Detection'
        WHEN scan_id LIKE 'PitDetection%' THEN 'Pit Detection'
        WHEN scan_id LIKE 'HoleDiameter%' THEN 'Hole Diameter'
        WHEN scan_id LIKE 'Offset%' THEN 'Offset'
        WHEN scan_id LIKE 'SurfaceRoughness%' THEN 'Surface Roughness'
        ELSE scan_id
    END as analysis_type,
    mmPerPixel,
    COUNT(*) as record_count
FROM customer01_data.silver.scans
GROUP BY scan_id, createdon, analysis_type, mmPerPixel
ORDER BY createdon DESC, analysis_type;

-- Validate analysis by type
SELECT * FROM customer01_data.gold.analysis_by_type;

In [ ]:
-- Gold Layer: Calibration Usage
-- Shows which calibrations are used, by which scans, and date coverage
CREATE OR REPLACE TABLE customer01_data.gold.calibration_usage AS
SELECT 
    sr.calib_id,
    COUNT(DISTINCT sr.scan_id) as scan_count,
    COLLECT_LIST(sr.scan_id) as scan_ids,
    MIN(sr.createdon) as earliest_scan_date,
    MAX(sr.createdon) as latest_scan_date,
    -- Join with calibrations table for calibration metadata
    MAX(c.calib_id) as calibration_record_id
FROM customer01_data.gold.scan_registry sr
LEFT JOIN customer01_data.silver.calibrations c
  ON sr.calib_id = c.calib_id
WHERE sr.calib_id IS NOT NULL
GROUP BY sr.calib_id
ORDER BY latest_scan_date DESC;

-- Validate calibration usage
SELECT * FROM customer01_data.gold.calibration_usage;

### Gold Layer Complete! 🎯

You now have 3 analytics tables ready for dashboards, reports, and decision-making:

| Table | Purpose | Use Case |
|-------|---------|----------|
| `gold.scan_registry` | Master dimension with calibration IDs | Scan lookup, audit trails, traceability |
| `gold.analysis_by_type` | Results grouped by analysis type | Defect vs pit vs roughness comparison |
| `gold.calibration_usage` | Calibration statistics & usage | Track which calibrations are used and for how long |

### Real Data Insights
✅ **Calibration Distribution**:
- Calib-4E07-XNJU_20250917_1255: 4 scans (9/17-9/29)
- Calib-436U-Y58K_20250912_0928: 1 scan (9/12)

### Next Steps
- **Build Dashboards** using Databricks SQL dashboards or connect to BI tools (Power BI, Tableau)
- **Create Alerts** on quality metrics or calibration age
- **Add More Gold Tables** as business needs evolve:
  - Defect trends by calibration
  - Scan quality scores with calibration traceability
  - Surface roughness statistics by analysis type
- **Document Lineage** - Track data from raw GelSight scans → Bronze → Silver → Gold